In [1]:
import gensim
import gensim.corpora as corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
stemmer = SnowballStemmer('english')
import numpy as np
np.random.seed(2018)
import nltk
import logging
import os
import requests as re
import pandas as pd
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from pprint import pprint
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from wordcloud import WordCloud
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /Users/kevins/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kevins/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/kevins/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
papers = pd.read_csv(os.getcwd().replace('scripts', 'data/papers/corpus_papers.csv'))



In [3]:
stop_words = stopwords.words('english')


In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [5]:
papers


,journal_id,volume_id,year,link,doi,date,pii,eid,scopus_id,citations,authors,affiliations,open_access
0,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104474,2023-08-01,S0309170823001094,2-s2.0-85161258968,8.516126e+10,0.0,"58307014400, 36008901600, 26635952600",60031268:60031268:60007989,True
1,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104475,2023-08-01,S0309170823001100,2-s2.0-85161636300,8.516164e+10,0.0,"53984251100, 35610240400","60104134:60104134, 60022461",True
2,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104476,2023-08-01,S0309170823001112,2-s2.0-85161667219,8.516167e+10,0.0,25822520800,60025063,True
3,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104477,2023-08-01,S0309170823001124,2-s2.0-85161661769,8.516166e+10,0.0,"57210424267, 36160282300, 7404296426",60018394:60018394:60018394,True
4,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104485,2023-08-01,S0309170823001203,2-s2.0-85161678744,8.516168e+10,0.0,"56708113100, 7202660185",60030447:60007566,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25847,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.020,2018-01-01,S1364815217304140,2-s2.0-85032268729,8.503227e+10,142.0,"34571084000, 55188785600, 33767605600, 5721638...","60016437:60012197:60012197:60012197, 60025278:...",True
25848,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.010,2018-01-01,S1364815216304005,2-s2.0-85032353211,8.503235e+10,54.0,"37111876100, 15069126500, 57189996027, 3574567...","60013141:60025182:60025182:60025182:60013141, ...",True
25849,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.08.002,2018-01-01,S1364815216309550,2-s2.0-85032684995,8.503268e+10,5.0,"7006615627, 49661600900, 15132591800, 66033150...","60021439:60030486, 109682148:60021439:60021439...",True
25850,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.011,2018-01-01,S136481521630843X,2-s2.0-85032736133,8.503274e+10,45.0,"56111768300, 37010006200, 43361846100, 5611302...",60068861:60020071:60068861:60024695:60102538:6...,True


In [6]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [16]:
papers_to_process = pd.DataFrame(columns = ['index', 'text'])

for i in range(4566,4568):
    papers_to_process = pd.concat([papers_to_process, pd.read_csv(os.getcwd().replace('scripts', f'data/text_corpus/text_corpus{i}.csv'))])

papers_to_process = papers_to_process.drop_duplicates().reset_index(drop = True)


In [17]:
papers_to_process

,index,text
0,22830,this paper concerns the prediction of the powe...
1,22831,due to the variability of the radiated signal ...
2,22832,pile foundation is one of the most commonly us...
3,22833,polymer grouting technology is a cost effectiv...
4,22834,stiffness of flexible connectors has great imp...
5,22835,developing design tools to make more informati...
6,22836,this paper proposes an efficient simulation dr...
7,22837,this study presents the development of an appr...
8,22838,the results of a computational fluid dynamics ...
9,22839,columns are the key load bearing elements in i...


In [18]:
processed_docs = papers_to_process['text'].map(preprocess)

In [19]:
id2word = corpora.Dictionary(processed_docs)

In [22]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [20]:
processed_docs

0    [paper, concern, predict, power, output, wave,...
1    [variabl, radiat, signal, sonar, target, passi...
2    [pile, foundat, common, foundat, offshor, coas...
3    [polym, grout, technolog, cost, effect, trench...
4    [stiff, flexibl, connector, great, impact, res...
5    [develop, design, tool, inform, avail, sooner,...
6    [paper, propos, effici, simul, drive, method, ...
7    [studi, present, develop, approxim, analyt, mo...
8    [result, comput, fluid, dynam, studi, blade, h...
9    [column, load, bear, element, infrastructur, c...
Name: text, dtype: object

In [23]:
# number of topics
num_topics = 4
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[bow_corpus]

[(0,
  '0.016*"wave" + 0.011*"pile" + 0.009*"load" + 0.009*"simul" + 0.009*"model" '
  '+ 0.009*"design" + 0.008*"result" + 0.008*"differ" + 0.007*"column" + '
  '0.007*"method"'),
 (1,
  '0.017*"wave" + 0.008*"model" + 0.007*"load" + 0.007*"design" + '
  '0.006*"studi" + 0.006*"effect" + 0.006*"result" + 0.006*"pile" + '
  '0.006*"show" + 0.006*"method"'),
 (2,
  '0.010*"load" + 0.008*"design" + 0.008*"wave" + 0.007*"result" + '
  '0.007*"differ" + 0.007*"model" + 0.007*"studi" + 0.006*"pipelin" + '
  '0.006*"pile" + 0.005*"show"'),
 (3,
  '0.013*"wave" + 0.012*"design" + 0.009*"model" + 0.008*"load" + '
  '0.007*"column" + 0.007*"ship" + 0.007*"differ" + 0.006*"method" + '
  '0.006*"result" + 0.005*"show"')]


In [24]:
pyLDAvis.enable_notebook()

In [25]:
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

In [26]:
#bag of words
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.007716 -0.003723       1        1  46.319975
3     -0.000008  0.006705       2        1  23.759694
2     -0.009665 -0.002979       3        1  21.498719
1      0.001957 -0.000003       4        1   8.421612, topic_info=           Term        Freq       Total Category  logprob  loglift
507        wave  424.000000  424.000000  Default  30.0000  30.0000
108      design  279.000000  279.000000  Default  29.0000  29.0000
133      effect  149.000000  149.000000  Default  28.0000  28.0000
1364    pipelin  108.000000  108.000000  Default  27.0000  27.0000
1117       pipe  126.000000  126.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
1115       pile   15.691458  250.340830   Topic4  -5.1003  -0.2953
1495  connector    9.436893  110.232429   Topic4  -5.6087   0.0164
116      differ   13.736700  215.944394   Topic4  -5.2333  -0.2806
1626       ship   11.615374  161.577068   Topic4  -5.4010  -0.1583
424       simul   11.547965  205.187712   Topic4  -5.4069  -0.4030

[336 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1201      1  0.549492  abaqus
1201      3  0.549492  abaqus
1202      1  0.353387  abrupt
1202      2  0.353387  abrupt
1202      3  0.353387  abrupt
...     ...       ...     ...
508       4  0.097371     wec
897       1  0.455844  weight
897       2  0.255273  weight
897       3  0.227922  weight
897       4  0.063818  weight

[804 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 2])

In [27]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.009394593513984821),
 (1, 0.01405563397152506),
 (2, 0.002081979111489905),
 (3, 0.0018450277753158883),
 (4, 0.014908928933304614),
 (5, 0.002981785786660923),
 (6, 0.014573853780429333),
 (7, 0.002081979111489905),
 (8, 0.00416395822297981),
 (9, 0.002981785786660923),
 (10, 0.00416395822297981),
 (11, 0.005348562314883217),
 (12, 0.002081979111489905),
 (13, 0.0320913738892993),
 (14, 0.01344062471308643),
 (15, 0.002981785786660923),
 (16, 0.01344062471308643),
 (17, 0.005348562314883217),
 (18, 0.008945357359982769),
 (19, 0.012491874668939428),
 (20, 0.01344062471308643),
 (21, 0.002081979111489905),
 (22, 0.037578374055939286),
 (24, 0.0006150092584386293),
 (25, 0.0013025311157816102),
 (26, 0.00416395822297981),
 (27, 0.0024600370337545174),
 (28, 0.026742811574416082),
 (29, 0.0018450277753158883),
 (30, 0.021083450957287586),
 (31, 0.01344062471308643),
 (32, 0.01344062471308643),
 (33, 0.002081979111489905),
 (34, 0.01405563397152506),
 (36, 0.002981785786660923),
 (

In [30]:
#TF IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=3, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.001*"connector" + 0.001*"modul" + 0.001*"stiff" + 0.001*"modular" + 0.001*"float" + 0.001*"flexibl" + 0.001*"wave" + 0.000*"porous" + 0.000*"pipelin" + 0.000*"multi"
Topic: 1 Word: 0.001*"pile" + 0.001*"collis" + 0.001*"ship" + 0.001*"wave" + 0.001*"wake" + 0.001*"risk" + 0.001*"pipelin" + 0.001*"pipe" + 0.001*"turbin" + 0.001*"nonlinear"
Topic: 2 Word: 0.001*"porous" + 0.001*"column" + 0.001*"blast" + 0.001*"submerg" + 0.001*"explos" + 0.001*"concret" + 0.001*"wave" + 0.001*"salp" + 0.001*"pipelin" + 0.001*"benchmark"


In [31]:
#tfidf
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.001014 -0.000115       1        1  60.494841
2      0.000785 -0.000258       2        1  29.610550
0      0.000229  0.000374       3        1   9.894609, topic_info=           Term       Freq      Total Category  logprob  loglift
1495  connector  13.000000  13.000000  Default  30.0000  30.0000
1575      modul  12.000000  12.000000  Default  29.0000  29.0000
1426      stiff  12.000000  12.000000  Default  28.0000  28.0000
2328     porous  16.000000  16.000000  Default  27.0000  27.0000
2342    submerg  15.000000  15.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
1364    pipelin   1.412400  21.801173   Topic3  -7.6693  -0.4235
273        load   1.264544  13.117918   Topic3  -7.7798  -0.0261
788      propel   1.295237  17.433892   Topic3  -7.7559  -0.2865
1787       fuel   1.286598  16.965350   Topic3  -7.7626  -0.2660
1892    optimis   1.290461  17.846849   Topic3  -7.7596  -0.3136

[142 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1472      1  0.626126    adjac
1472      2  0.268340    adjac
1472      3  0.089447    adjac
1480      1  0.611617   assign
1480      2  0.262121   assign
...     ...       ...      ...
2027      2  0.196664  weather
2027      3  0.065555  weather
897       1  0.610783   weight
897       2  0.305391   weight
897       3  0.076348   weight

[306 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [85]:
threshold = 25

In [86]:
for i in range(26474):
    tmp = buzzwords_in_papers[buzzwords_in_papers['paper_id']==i]
    
    grouped = tmp.groupby('buzz_id').count()
    
#     print(grouped)
    
    for j in range(grouped.shape[0]):
        if grouped.iat[j,0] >=threshold:
            lots_of_words.loc[len(lots_of_words.index)] = [i, grouped.iloc[j].name, grouped.iat[j,0]]

In [74]:
lots_of_words

,paper_id,buzz_id,count
0,1,18,102
1,3,6,65
2,7,6,59
3,9,37,62
4,19,0,99
...,...,...,...
3441,26466,24,51
3442,26468,10,75
3443,26470,0,90
3444,26470,6,83


In [76]:
len(set(lots_of_words['paper_id'].tolist()))

2225